In [1]:
import pandas as pd
import numpy as np
import cython
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df = pd.read_csv("C:/Users/iwama/Desktop/Osobiste/MSc/put-msc/Endometriosis/endometriosis.csv")
X = df.drop(columns=['target'])
y = df['target'].astype('int')
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [2]:
from CalculateMetrics import calculate_all_metrics
from EnderClassifier import EnderClassifier

print("ENDER CLASSIFIER NORMAL")
ender = EnderClassifier(verbose=True, n_rules=5, use_gradient=False)
ender.fit(X_train, y_train)

ENDER CLASSIFIER NORMAL
Default rule: [0.4979062213548282, -0.4979062213548282]
####################################################################################
Rule: 1
	sex_pain in [-99999999999999999, 1.5]
	defecation_pain_during in [-99999999999999999, 1.5]
	chronic_pain_level in [-99999999999999999, 3.5]
	family_endo in [-99999999999999999, 0.5]
	menstruation_lenght in [-99999999999999999, 27.0]
	stomach_ache_apart in [0.5, 99999999999999999]
	abdominal_pressure_apart in [0.5, 99999999999999999]
	cycle in [-99999999999999999, 45.0]
=> vote for class 0 with weight 0.5681012197701224
[0.5681012197701224, -0.5681012197701224]

####################################################################################
Rule: 2
	sex_pain_level in [3.5, 99999999999999999]
	chronic_pain_level in [0.5, 99999999999999999]
	menstruation_pain in [5.5, 99999999999999999]
	age in [23.5, 99999999999999999]
	smoking_time in [-99999999999999999, 23.5]
	height in [157.0, 99999999999999999]
	weight in [

In [3]:
y_train_pred = ender.predict(X_train)
y_test_pred = ender.predict(X_test)

train_metrics = calculate_all_metrics(y_train, y_train_pred)
test_metrics = calculate_all_metrics(y_test, y_test_pred)

y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

print("Normal Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Normal Ender Classifier
Accuracy: 0.82

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.84      0.95      0.89       150
           1       0.71      0.42      0.53        48

    accuracy                           0.82       198
   macro avg       0.77      0.68      0.71       198
weighted avg       0.81      0.82      0.80       198

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    142                 8                     # True Negative (TN), False Positive (FP)
Endometriosis       28                  20                    # False Negative (FN), True Positive (TP)


In [4]:
from EnderClassifierBounded import EnderClassifier

print("ENDER CLASSIFIER BOUNDED")
ender = EnderClassifier(verbose=True, n_rules=5, use_gradient=False)
ender.fit(X_train, y_train)

ENDER CLASSIFIER BOUNDED
Initializing for rule...
Default rule: [0.4979062213548282, -0.4979062213548282]
####################################################################################
Rule: 1
Default Rule: MSE = 0.18340558976477644
Initializing for rule...
	sex_pain in [-99999999999999999, 1.5]
	defecation_pain_during in [-99999999999999999, 1.5]
	chronic_pain_level in [-99999999999999999, 3.5]
	family_endo in [-99999999999999999, 0.5]
	menstruation_lenght in [-99999999999999999, 27.0]
	stomach_ache_apart in [0.5, 99999999999999999]
	abdominal_pressure_apart in [0.5, 99999999999999999]
	cycle in [-99999999999999999, 45.0]
=> vote for class 0 with weight 0.5681012197701224
[0.5681012197701224, -0.5681012197701224]

####################################################################################
Rule: 2
Default Rule: MSE = 0.18340558976477644
Rule 0: MSE = 0.17522474124544135
Initializing for rule...
	sex_pain_level in [3.5, 99999999999999999]
	chronic_pain_level in [0.5, 9999

In [6]:
from EnderClassifierBounded import EnderClassifier

print("ENDER CLASSIFIER BOUNDED")
ender = EnderClassifier(verbose=True, n_rules=5, use_gradient=False)
ender.fit(X_train, y_train)

ENDER CLASSIFIER BOUNDED
Initializing for rule...
Default rule: [0.4979062213548282, -0.4979062213548282]
####################################################################################
Rule: 1
Default Rule: MSE = 0.18340558976477644
Initializing for rule...
Lower bound 0.0987814654481325 is less than empirical risk 0.13033206194634442 for rule with 1 conditions. Continuing search.
Lower bound 0.07017664694919433 is less than empirical risk 0.114488401460526 for rule with 2 conditions. Continuing search.
Lower bound 0.03790438766813349 is less than empirical risk 0.09788993367423247 for rule with 3 conditions. Continuing search.
Lower bound 0.024189393939395152 is less than empirical risk 0.0888898593793885 for rule with 4 conditions. Continuing search.
Lower bound 0.019687279372068205 is less than empirical risk 0.08665693221707065 for rule with 5 conditions. Continuing search.
Lower bound 0.01517208578346724 is less than empirical risk 0.08452010332641013 for rule with 6 conditi

In [5]:
y_train_pred = ender.predict(X_train)
y_test_pred = ender.predict(X_test)

train_metrics = calculate_all_metrics(y_train, y_train_pred)
test_metrics = calculate_all_metrics(y_test, y_test_pred)

y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

print("Bounded Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Bounded Ender Classifier
Accuracy: 0.84

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       150
           1       0.77      0.50      0.61        48

    accuracy                           0.84       198
   macro avg       0.82      0.73      0.75       198
weighted avg       0.84      0.84      0.83       198

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    143                 7                     # True Negative (TN), False Positive (FP)
Endometriosis       24                  24                    # False Negative (FN), True Positive (TP)


In [2]:
from CalculateMetrics import calculate_all_metrics
from EnderClassifierModified import EnderClassifier

print("EnderClassifierModified")
enderM = EnderClassifier(verbose=False, n_rules=20, use_gradient=False)
enderM.fit(X_train, y_train)

EnderClassifierModified


In [3]:
y_train_pred = enderM.predict(X_train)
y_test_pred = enderM.predict(X_test)

train_metrics = calculate_all_metrics(y_train, y_train_pred)
test_metrics = calculate_all_metrics(y_test, y_test_pred)

y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

print("Modified Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Modified Ender Classifier
Accuracy: 0.81

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       150
           1       0.64      0.52      0.57        48

    accuracy                           0.81       198
   macro avg       0.75      0.71      0.73       198
weighted avg       0.80      0.81      0.81       198

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    136                 14                    # True Negative (TN), False Positive (FP)
Endometriosis       23                  25                    # False Negative (FN), True Positive (TP)


In [12]:
for n in range(15, 45):
    ender = EnderClassifier(verbose=False, n_rules=n, use_gradient=False)
    ender.fit(X_train, y_train)
    y_train_pred = ender.predict(X_train)
    y_train_pred = np.array([np.argmax(pred) for pred in y_train_pred])
    y_test_pred = ender.predict(X_test)
    y_pred = np.array([np.argmax(pred) for pred in y_test_pred])
    print(f"EnderClassifier with {n} rules")
    print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred):.2f}")
    print("================================")

EnderClassifier with 15 rules
Test Accuracy: 0.82
Train Accuracy: 0.94
EnderClassifier with 16 rules
Test Accuracy: 0.82
Train Accuracy: 0.94
EnderClassifier with 17 rules
Test Accuracy: 0.83
Train Accuracy: 0.95
EnderClassifier with 18 rules
Test Accuracy: 0.83
Train Accuracy: 0.95
EnderClassifier with 19 rules
Test Accuracy: 0.83
Train Accuracy: 0.96
EnderClassifier with 20 rules
Test Accuracy: 0.83
Train Accuracy: 0.96
EnderClassifier with 21 rules
Test Accuracy: 0.83
Train Accuracy: 0.96
EnderClassifier with 22 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 23 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 24 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 25 rules
Test Accuracy: 0.82
Train Accuracy: 0.97
EnderClassifier with 26 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 27 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 28 rules
Test Accuracy: 0.83
Train Accuracy: 0.98
EnderC

In [6]:
from time import time
ender = EnderClassifier(verbose=False, n_rules=41, use_gradient=False)
start_time = time()
ender.fit(X_train, y_train)
# y_train_pred = ender.predict(X_train)
# y_train_pred = np.array([np.argmax(pred) for pred in y_train_pred])
y_test_pred = ender.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in y_test_pred])
print(f"Training time for 14 rules: {time() - start_time:.2f} seconds")

print("Modified Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Training time for 14 rules: 0.51 seconds
Modified Ender Classifier
Accuracy: 0.84

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       151
           1       0.75      0.50      0.60        48

    accuracy                           0.84       199
   macro avg       0.80      0.72      0.75       199
weighted avg       0.83      0.84      0.83       199

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    143                 8                     # True Negative (TN), False Positive (FP)
Endometriosis       24                  24                    # False Negative (FN), True Positive (TP)


In [6]:
# from EnderRegressor import EnderRegressor
# from CalculateMetrics import calculate_all_metrics

# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# df = pd.read_csv("/home/maciej/Desktop/Maciej/endometriosis/endometriosis.csv")
# X = df.drop(columns=['target'])
# y = df['target'].astype('int')
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# ender = EnderRegressor(verbose=False)
# ender.fit(X_train, y_train)

In [7]:
# ender.evaluate_all_rules()
# ender.rules

In [8]:
# y_train_pred = ender.predict(X_train)
# y_test_pred = ender.predict(X_test)

# train_metrics = calculate_all_metrics(y_train, y_train_pred)
# test_metrics = calculate_all_metrics(y_test, y_test_pred)

In [9]:
# y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

# print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
# print("Classification Report - Test:")
# print(classification_report(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# labels = ['No Endometriosis', 'Endometriosis']
# print("Confusion Matrix:")
# print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
# print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
# print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

In [10]:
# from EnderClassifier import EnderClassifier as ECOG

In [11]:
# ender2 = ECOG(verbose=False)
# ender2.fit(X_train, y_train)

In [12]:
# ender2.evaluate_all_rules()

In [13]:
# ender = ender2

In [14]:
# y_train_pred = ender.predict(X_train)
# y_test_pred = ender.predict(X_test)
# train_metrics = calculate_all_metrics(y_train, y_train_pred)
# test_metrics = calculate_all_metrics(y_test, y_test_pred)

In [15]:
# y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

# print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
# print("Classification Report - Test:")
# print(classification_report(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# labels = ['No Endometriosis', 'Endometriosis']
# print("Confusion Matrix:")
# print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
# print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
# print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")